In [2]:
import pandas as pd
import numpy as np
import time
import os
import re
import progressbar

In [3]:
import transliterate
from transliterate import translit, get_available_language_codes

In [4]:
#path to data
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/'

In [5]:
df_red_results = pd.read_csv(PATH_TO_DATA+'uk_red_links_results_all_candidates.csv', encoding = 'UTF-8')

In [6]:
df_red_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3196 entries, 0 to 3195
Data columns (total 2 columns):
red_link_name    3196 non-null object
en_similar       3196 non-null object
dtypes: object(2)
memory usage: 50.0+ KB


In [7]:
df_red_results.head()

,red_link_name,en_similar
0,Каунт Бейсі,"[(0.081, 'The Count Basie Story'), (0.076, 'Th..."
1,аль-Каїм,"[(0.031, 'Ahmad al-Muqtadir'), (0.03, 'Cai Xia..."
2,королева,"[(0.033, 'List of consorts of the monarch of I..."
3,Prentice Hall,"[(0.078, 'Chemical tanker'), (0.067, 'Break of..."
4,Tokar.ua,"[(0.05, 'List of virtual communities with more..."


In [ ]:
# text = "Outlawz"
# print(translit(text, 'uk', reversed=True))

In [8]:
list_of_red_links = df_red_results['red_link_name'].tolist()

In [9]:
transliterated_rl = []
for redlink in list_of_red_links:
    latinized = translit(redlink, 'uk', reversed=True)
    transliterated_rl.append(latinized)

In [10]:
transliterated_rl[1]

"al'-Kayim"

In [11]:
def extract_list(s):
    return [x.split(',')[1].strip("', ,\"") for x in s[2:-2].split('), (')]

In [12]:
all_results = {}
all_results_list = [[]] * df_red_results.shape[0]
for i in range(df_red_results.shape[0]):
    if df_red_results['en_similar'][i] == '[]':
        all_results[df_red_results['red_link_name'][i]] = []
        all_results_list[i] = []
    else:
        res = extract_list(df_red_results['en_similar'][i])
        all_results[df_red_results['red_link_name'][i]] = res
        all_results_list[i] = res

In [15]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [33]:
#min_indx = np.argmin(np.array([levenshteinDistance(transliterated_rl[0], x) for x in all_results_list[0]]))
#all_results_list[0][min_indx]

In [18]:
matched_red_links = []
for row in range(len(transliterated_rl)):
    if all_results_list[row] == []:
        matched_red_links.append('')
    else:
        min_indx = np.argmin(np.array([levenshteinDistance(transliterated_rl[row], x) for x in all_results_list[row]]))
        matched = all_results_list[row][min_indx]
        matched_red_links.append(matched)

In [24]:
df_matched_red_links = pd.DataFrame({'matched_red_links': matched_red_links})

In [25]:
df_matched_red_links

,matched_red_links
0,Count Basie
1,Glamis
2,Coronet
3,Prentice Hall
4,Tsarina
5,Monthly Notices of the Royal Astronomical Society
6,Livingston
7,Dutch Canadians
8,Robin Hood's Bay
9,Romani people in Bulgaria


In [26]:
df_red_results = pd.read_csv(PATH_TO_DATA+'uk_red_links_results_lighter.csv', encoding = 'UTF-8')

In [28]:
df_red_links_with_ground_truth = df_red_results[['red_link_name', 'ground_truth']]

In [39]:
# save results to file
uk_red_links_transliteration_results.to_csv(PATH_TO_DATA+'uk_red_links_transliteration_results.csv', header=True, index=False, encoding = 'UTF-8')

In [ ]:
# think on another edit distance

In [16]:
def intersectionDistance(s1, s2):
    f = set(s1).intersection(set(s2))
    return len(f)

In [18]:
matched_red_links = []
for row in range(len(transliterated_rl)):
    if all_results_list[row] == []:
        matched_red_links.append('')
    else:
        max_indx = np.argmax(np.array([intersectionDistance(transliterated_rl[row], x) for x in all_results_list[row]]))
        matched = all_results_list[row][max_indx]
        matched_red_links.append(matched)

In [19]:
df_matched_red_links = pd.DataFrame({'matched_red_links': matched_red_links})

In [20]:
df_matched_red_links

,matched_red_links
0,Billy Mitchell (jazz musician)
1,Al-Mu'tamid ibn Abbad
2,Drama Desk Award for Outstanding Revival
3,Prentice Hall
4,Tokyu Kogyo Kurogane
5,Monthly Notices of the Royal Astronomical Society
6,Royalist (Spanish American independence)
7,List of countries by HIV/AIDS adult prevalence...
8,List of multiple Winter Olympic medalists
9,National Commission for the Development of Ind...


In [21]:
df_red_results = pd.read_csv(PATH_TO_DATA+'uk_red_links_results_lighter.csv', encoding = 'UTF-8')
df_red_links_with_ground_truth = df_red_results[['red_link_name', 'ground_truth']]
uk_red_links_transliteration_results = pd.concat([df_red_links_with_ground_truth, df_matched_red_links], axis=1)

In [22]:
df_red_results = uk_red_links_transliteration_results.apply(lambda x: x.str.strip())
len(df_red_results)

3196

In [26]:
df_red_results

,red_link_name,ground_truth,matched_red_links
0,Каунт Бейсі,Count Basie,Billy Mitchell (jazz musician)
1,аль-Каїм,"?? Al-Qa'im (town), Al-Qa'im (Cairo), Al-Qa'im...",Al-Mu'tamid ibn Abbad
2,королева,"?? gyne, monarch, queen regnant, queen (chess)...",Drama Desk Award for Outstanding Revival
3,Prentice Hall,Prentice Hall,Prentice Hall
4,Tokar.ua,NaN,Tokyu Kogyo Kurogane
5,Monthly Notices of the Royal Astronomical Society,Monthly Notices of the Royal Astronomical Society,Monthly Notices of the Royal Astronomical Society
6,Світ (видавництво),NaN,Royalist (Spanish American independence)
7,Дубович Іван Андрійович,NaN,List of countries by HIV/AIDS adult prevalence...
8,помірні повітряні маси,NaN,List of multiple Winter Olympic medalists
9,Доповідь про торгівлю людьми,Trafficking in Persons Report,National Commission for the Development of Ind...


In [25]:
df_red_results = uk_red_links_transliteration_results.apply(lambda x: x.str.replace('_', ' '))

In [27]:
evaluation = []
for row in range(len(df_red_results)):
    if df_red_results['ground_truth'][row] == df_red_results['matched_red_links'][row]:
        evaluation.append('TP')
    elif pd.isnull(df_red_results['ground_truth'][row]) & pd.isnull(df_red_results['matched_red_links'][row]):
        evaluation.append('TN')
    elif pd.isnull(df_red_results['ground_truth'][row]) & pd.notnull(df_red_results['matched_red_links'][row]):
        evaluation.append('FP')
    elif pd.notnull(df_red_results['ground_truth'][row]) & pd.isnull(df_red_results['matched_red_links'][row]):
        evaluation.append('FN')
    elif df_red_results['ground_truth'][row] != df_red_results['matched_red_links'][row]:
        evaluation.append('FP')

In [28]:
evaluation = pd.DataFrame({'evaluation': evaluation})

In [29]:
uk_red_links_transliteration_evaluation = pd.concat([df_red_results, evaluation], axis=1)

In [30]:
res = uk_red_links_transliteration_evaluation['evaluation'].value_counts()

In [33]:
tp = res['TP']
#tn = res['TN']
fp = res['FP']
#fn = res['FN']

In [34]:
precision = tp/(tp + fp)
print('precision =', precision)

precision = 0.43460575719649563


In [35]:
recall = tp/(tp + 0)
print('recall =', recall)

recall = 1.0


In [36]:
f1_score = (2*precision*recall)/(precision+recall)
print('f1_score =', f1_score)

f1_score = 0.6058887677208288
